## Red neuronal utilizando Keras

Ahora construiremos una red neuronal utilizando la libreria de Keras que imprementa tensorflow.
Utilizaremos la base de datos sin valores faltantes y con las modificaciones mencionadas previamente para entrenar el modelo. El modelo se elegira mediante validación cruzada, para probar su efectividad.

### Cargando librerias necesarias

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
#tf.enable_eager_execution()
import numpy as np
import pandas as pd
import keras
import os
import functools


Cargamos la base de datos desde Github.

In [ ]:
train = pd.read_csv('https://github.com/DavidChaMun/TAE_FINAL/raw/master/data/dataset_tae_final_no_na_mod.csv')
test = pd.read_csv('https://github.com/DavidChaMun/TAE_FINAL/raw/master/data/test_tae_no_na_mod.csv')

Observemos la estructura de los datos:

In [ ]:
#Chequeo de tipos de las variables
train.dtypes

age                  int64
workclass         category
fnlwgt               int64
education         category
marital_status    category
ocupation         category
ethnicity         category
gender            category
capital_gain         int64
capital_loss         int64
hours_per_week       int64
native_country    category
income                int8
dtype: object

## Transformación de la base de datos a un formato conveniente

Vemos que tenemos variables categóricas, debemos transformarlas a variables dummies para poder crear la red neuronal:

In [ ]:
catego_columns = ['education', 'workclass', 'marital_status', 
             'ethnicity', 'income','gender',
                  'native_country', 'ocupation']
#Transformamos variables object a categoricas
for col in catego_columns:
    train[col] = pd.Categorical(train[col])
    test[col] = pd.Categorical(test[col])

#Transformamos nuestra variable de objetivo para la clasificacion
train[label] = train[label].cat.codes
test[label] = test[label].cat.codes

#Creamos variables dummies con las categoricas
train_dataset=pd.get_dummies(train)
test_dataset=pd.get_dummies(test)

Chequeamos que las variables de nuestro conjunto de entrenamiento sean las mismas que las de validación

In [ ]:
for elem in list(train_dataset.columns):
    if not((elem in list(test_dataset.columns))):
        print(elem)
#Dado que no imprime nada, las dos bases tienen las mismas variables dummies.

In [ ]:
columns = train_dataset.columns
columns

Index(['age', 'fnlwgt', 'capital_gain', 'capital_loss', 'hours_per_week',
       'income', 'workclass_ Federal-gov', 'workclass_ Local-gov',
       'workclass_ Private', 'workclass_ Self-emp-inc',
       'workclass_ Self-emp-not-inc', 'workclass_ State-gov',
       'workclass_ Without-pay', 'education_ 10th', 'education_ 11th',
       'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th',
       'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm',
       'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate',
       'education_ HS-grad', 'education_ Masters', 'education_ Preschool',
       'education_ Prof-school', 'education_ Some-college',
       'marital_status_ Divorced', 'marital_status_ Never-married',
       'marital_status_ Separated', 'marital_status_ Widowed',
       'marital_status_Married', 'ocupation_ Adm-clerical',
       'ocupation_ Armed-Forces', 'ocupation_ Craft-repair',
       'ocupation_ Exec-managerial', 'ocupation_ Farming-fishi

In [ ]:
#train_dataset.to_csv('train_dummies.csv')

# Creando la red neuronal

Para la construcción de la red neuronal se utilizo el modelo *sequential* de Keras, la cuál ajusta una red neuronal (utilizando ciertos métodos que describiremos más adelante), dicha función se entrena utilizando *.fit()*, este método recibe un conjunto de entrenamiento y permite introducir un conjunto de validación, por lo que en cada iteración de entrenamiento se puede ver la precisión de la predicción para ambos conjuntos de datos. 

Se eligió como criterio de elección del modelo aquel que maximizara la precisión en las predicciones de la variables *income*. Para la elección del modelo se variaron los siguientes atributos:

- Número de capas y neuronas dentro de *sequential*: aumentar el numero de neuronas a más de dos no mejoró los resultados, el número de neuronas mejoraba los resultados al aumentarlas por encimas de 10, al final se eligieron 2 capas internas con 30 y 20 neuronas respectivamentes.

- Método de activación de la neurona de salida: se probo "softmax" que según la documentación de internet es eficiente en clasificación de 2 o multiples variables categóricas. Sin embargo, este mostro precisiones desastrozas por debajo de 0.30 en general. Por lo que se utilizo "sigmoid".

- Optimizador: se probaron *SGD* (precisiones de hasta 0.83), *Adam* y *RMSprop*, variando el __learning rate (LR)__ en todos ellos, se encontro que *Adam* y *RMSprop* llegaban a resultados equivalente de hasta 0.85 de precisión, los LR entre 0.001 y 0.1 convergian siempre a una precisión de 0.85, por lo que se eligió 0.1 al entrenar la red más rapidamente.

- Número de epochs: estos son definimos al usar *model.fit()*. Aumentarlos o disminuirlos no impacto mucho el modelo final, la neurona convergia para el 10avo epoch, se eligio 20 para visualizar un comportamiento más prolongado.

### A continuación se muestra la red neuronal final construida:

#### Definición de la red neuronal:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import adam

#Función que crea la red
def get_compiled_model():
  #Modelo sequencial, posee las neuronas
  model = Sequential([
    Dense(40, input_shape=(x_train.shape[1:])),
    Activation('relu'),
    Dense(20),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid'), #última neuronal
  ])
  
  #optimizador
  opt= adam(lr=0.01,beta_1=0.9, beta_2=0.9)
  
  #definición del copiador
  model.compile(optimizer=opt,
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

#### Escalado y definición de variables de apoyo:

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#Definimos las variables de entrenamiento y las objetivo para entrenar el modelo
x_train = train_dataset.copy(deep=True)
x_val = test_dataset.copy(deep=True)
y_train = x_train.pop('income').values
y_val = x_val.pop('income').values

#Escalamos los datos
scaler = MinMaxScaler() #definimos nuestro escalador, nos será útil a la hora de convertir los datos para hacer las predicciones.
scaler.fit(x_train)    
x_train =  scaler.transform(x_train)
x_val = scaler.transform(x_val)

### Entrenamiento y validación de la red neuronal:

In [ ]:
#Obtenemos nuestro modelo
model = get_compiled_model()

#Entrenamos el modelo.
model.fit(x=x_train, y=y_train,validation_data=(x_val,y_val),epochs = 20,batch_size=20,shuffle=True)

Train on 30160 samples, validate on 15059 samples
Epoch 1/20
30160/30160 [==============================] - 4s 121us/step - loss: 0.3632 - acc: 0.8313 - val_loss: 0.3498 - val_acc: 0.8375
Epoch 2/20
30160/30160 [==============================] - 3s 83us/step - loss: 0.3440 - acc: 0.8404 - val_loss: 0.3464 - val_acc: 0.8335
Epoch 3/20
30160/30160 [==============================] - 3s 84us/step - loss: 0.3412 - acc: 0.8440 - val_loss: 0.3353 - val_acc: 0.8447
Epoch 4/20
30160/30160 [==============================] - 3s 84us/step - loss: 0.3386 - acc: 0.8457 - val_loss: 0.3348 - val_acc: 0.8465
Epoch 5/20
30160/30160 [==============================] - 2s 82us/step - loss: 0.3350 - acc: 0.8473 - val_loss: 0.3400 - val_acc: 0.8443
Epoch 6/20
30160/30160 [==============================] - 3s 84us/step - loss: 0.3353 - acc: 0.8488 - val_loss: 0.3373 - val_acc: 0.8475
Epoch 7/20
30160/30160 [==============================] - 3s 84us/step - loss: 0.3346 - acc: 0.8487 - val_loss: 0.3323 - val_ac

Nuestro modelo final tiene una precisión del **~85%** en los datos de validación. 

### Demo de nuevas predicciones

Ahora construiremos una función que reciba una base numpy con los datos a predecir, construya las variables dummies, las escale y sea predicho por el modelo final.

Para conseguir la misma estructura de la base de datos con la que se entreno el modelo, basta con crear un pd.data_frame con valores de un _input valido_, y añadir las columnas de la df con dummies que falten con valores con cero. Luego se escala con el objeto _scaler_ con el que se escaló la base original. 

Una clase que realiza este proceso cargando la red neuronal y el scaler se presenta a continuación:

In [ ]:
#Librerias necesarias:
from sklearn.externals import joblib
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

import joblib
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential


class neural_network_mod():

    scaler =None
    nn_model = None
    train_cols = ['age', 'fnlwgt', 'capital_gain', 'capital_loss', 'hours_per_week',
       'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Private',
       'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc',
       'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 10th',
       'education_ 11th', 'education_ 12th', 'education_ 1st-4th',
       'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th',
       'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors',
       'education_ Doctorate', 'education_ HS-grad', 'education_ Masters',
       'education_ Preschool', 'education_ Prof-school',
       'education_ Some-college', 'marital_status_ Divorced',
       'marital_status_ Never-married', 'marital_status_ Separated',
       'marital_status_ Widowed', 'marital_status_Married',
       'ocupation_ Adm-clerical', 'ocupation_ Armed-Forces',
       'ocupation_ Craft-repair', 'ocupation_ Exec-managerial',
       'ocupation_ Farming-fishing', 'ocupation_ Handlers-cleaners',
       'ocupation_ Machine-op-inspct', 'ocupation_ Other-service',
       'ocupation_ Priv-house-serv', 'ocupation_ Prof-specialty',
       'ocupation_ Protective-serv', 'ocupation_ Sales',
       'ocupation_ Tech-support', 'ocupation_ Transport-moving',
       'ethnicity_ Amer-Indian-Eskimo', 'ethnicity_ Asian-Pac-Islander',
       'ethnicity_ Black', 'ethnicity_ Other', 'ethnicity_ White',
       'gender_ Female', 'gender_ Male', 'native_country_ Mexico',
       'native_country_ United-States', 'native_country_other']
    catego_columns = ['education', 'workclass', 'marital_status', 
             'ethnicity','gender',
                  'native_country', 'ocupation']
    
    def __init__(self):
        #Carga el scaler y la red neuronal:
        self.scaler = joblib.load('scaler.save')
        self.nn_model = load_model('my_model.h5')
        
    def predict(self, data):
      #Esta función s eutiliza para predecir, recibe data un pd_dataframe
      
      #Transformamos la base de datos
      data = self.tidy_data(data)
      
      predictions = self.nn_model.predict(data)>0.5
      predictions=np.where(predictions==0, "<=50k", predictions)
      predictions=np.where(predictions=='True', ">50k", predictions)
      return(predictions)
      
    def tidy_data(self,data):
      
      #Transformamos variables object a categoricas
      for col in self.catego_columns:        
        data[col] = pd.Categorical(data[col])
      
      #Creamos variables dummies con las categoricas
      data_d=pd.get_dummies(data)
      
      #Rellenamos con las variables dummies
      missing_cols = set(self.train_cols) - set(data_d.columns)
      
      for c in missing_cols:
        data_d[c] = 0
        
      # Ensure the order of column in the test set is in the same order than in train set
      data_d = data_d[self.train_cols]
      
      #Escalamos los datos
      data_d = self.scaler.transform(data_d)    
      return(data_d)
      
 


ob = neural_network_mod()
ob.predict(new_pred)
#model.predict(x_train[[1,2],:])>0.5
#y_train

### Guardando el modelo:

In [ ]:


from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
model.save('my_model.h5')

# Deletes the existing model
#del model  

# Returns a compiled model identical to the previous one
model2 = load_model('my_model.h5')

In [ ]:
train_cols

Index(['age', 'fnlwgt', 'capital_gain', 'capital_loss', 'hours_per_week',
       'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Private',
       'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc',
       'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 10th',
       'education_ 11th', 'education_ 12th', 'education_ 1st-4th',
       'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th',
       'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors',
       'education_ Doctorate', 'education_ HS-grad', 'education_ Masters',
       'education_ Preschool', 'education_ Prof-school',
       'education_ Some-college', 'marital_status_ Divorced',
       'marital_status_ Never-married', 'marital_status_ Separated',
       'marital_status_ Widowed', 'marital_status_Married',
       'ocupation_ Adm-clerical', 'ocupation_ Armed-Forces',
       'ocupation_ Craft-repair', 'ocupation_ Exec-managerial',
       'ocupation_ Farming-fishing', 'ocup

In [ ]:
from sklearn.externals import joblib
scaler_filename = "scaler.joblib"
joblib.dump(scaler, scaler_filename) 


# And now to load...
scaler2 = joblib.load(scaler_filename) 
# scaler = joblib.load(scaler_filename) 
type(scaler2)

sklearn.preprocessing.data.MinMaxScaler

In [1]:
import joblib
from joblib import load
test2 = load("testSVM2.joblib")
test1 = load("testSVM1.joblib")

In [2]:
test1.head(10)

,age,fnlwgt,capital_gain,capital_loss,hours_per_week,education,workclass,marital_status,ocupation,ethnicity,gender,native_country
0,40.0,160323.0,0.0,0.0,0.0,Doctorate,Private,Married,Machine-op-inspct,White,Female,United-States


In [3]:
test2.head(10)

,age,fnlwgt,capital_gain,capital_loss,hours_per_week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,...,ethnicity_ Asian-Pac-Islander,ethnicity_ Black,ethnicity_ Other,ethnicity_ White,gender_ Female,gender_ Male,native_country_ Mexico,native_country_ United-States,native_country_other,Unnamed
30159,52,287927,15024,0,40,0,0,0,1,0,...,0,0,0,1,1,0,0,1,0,30159


In [4]:
model = load("svm_model.joblib")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator SVC from version 0.21.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
for col in test2.columns:
    test2[col].values[:] = 0
model.predict(test2)

array([0], dtype=int64)